### Set up

#### 1. Set  up  accounts and role

In [1]:
#!pip install sagemaker==1.39.0

In [2]:
import sagemaker
import boto3

sagemaker_session = sagemaker.Session()
account_id =  boto3.client('sts').get_caller_identity().get('Account')
region = boto3.session.Session().region_name


#role = sagemaker.get_execution_role()
role="arn:aws:iam::{}:role/service-role/AmazonSageMaker-ExecutionRole-20190118T115449".format(account_id)


#### 2. Setup image and instance type

In [3]:
pytorch_custom_image_name="ppi-extractor:gpu-1.0.0-201910130520"
instance_type = "ml.p3.8xlarge" 

In [4]:
docker_repo = "{}.dkr.ecr.{}.amazonaws.com/{}".format(account_id, region, pytorch_custom_image_name)

#### 3. Configure train/ test and validation datasets

In [5]:
bucket = "aegovan-data"

In [6]:
#trainfile = "s3://{}/aimed/AIMedtrain.json".format(bucket)
trainfile_plain = "s3://{}/aimed/AIMedFull.json".format(bucket)
trainfile="s3://{}/aimed/AIMedFull_preprocessed.json".format(bucket)

valfile="s3://{}/aimed/AIMedval_preprocessed.json".format(bucket)

pretrained_bert="s3://{}/embeddings/bert/".format(bucket)


s3_output_path= "s3://{}/results/".format(bucket)
s3_code_path= "s3://{}/aimed_bert_code".format(bucket)

### Start training

In [7]:
commit_id = "2daad1647a714b50dd696a02df3024c08be39585"

In [8]:
git_config = {'repo': 'https://github.com/elangovana/PPI-typed-relation-extractor.git',
              'branch': 'master',
              'commit': commit_id
             }

In [9]:
pub_inputs = {
    "train" : trainfile,
    "PRETRAINED_BIOBERT" : pretrained_bert
}

In [10]:
plain_inputs = {
    "train" : trainfile_plain,
    "PRETRAINED_BIOBERT" : pretrained_bert
}

In [11]:
BertNetworkFactoryhyperparameters = {
    "dataset":"PpiAimedDatasetPreprocessedFactory",
    "network" :"RelationExtractorBioBertFactory",
    "trainfile":trainfile.split("/")[-1],
    "batchsize": "8",
    "accumulation_steps" : "8",
    "epochs" : "1000",   
    "log-level" : "INFO",
    "learningrate":.00001,
    "earlystoppingpatience":20,
    "commit_id" : commit_id
}

In [12]:
BertNetworkFactoryhyperparameters_plain = {
    "dataset":"PpiAimedDatasetFactory",
    "network" :"RelationExtractorBioBertFactory",
    "trainfile":trainfile_plain.split("/")[-1],
    "batchsize": "8",
    "accumulation_steps" : "8",
    "epochs" : "1000",   
    "log-level" : "INFO",
    "learningrate":.00001,
    "earlystoppingpatience":20,
    "commit_id" : commit_id
}

In [13]:
LstmBertNetworkFactoryhyperparameters_plain = {
    "dataset":"PpiAimedDatasetFactory",
    "network" :"RelationExtractorBertBiLstmNetworkNoPosFactory",
    "trainfile":trainfile_plain.split("/")[-1],
    "batchsize": "64",
    "accumulation_steps" : "1",
    "epochs" : "1000",   
    "log-level" : "INFO",
    "learningrate":.00001,
    "earlystoppingpatience":20,
    "commit_id" : commit_id
}

In [14]:
metric_definitions = [{"Name": "TrainLoss",
                     "Regex": "###score: train_loss### (\d*[.]?\d*)"}
                    ,{"Name": "ValidationLoss",
                     "Regex": "###score: val_loss### (\d*[.]?\d*)"}
                    ,{"Name": "TrainFScore",
                     "Regex": "###score: train_fscore### (\d*[.]?\d*)"}
                   ,{"Name": "ValidationFScore",
                     "Regex": "###score: val_fscore### (\d*[.]?\d*)"}
                    ]

In [15]:
!git log -1 | head -1
!git log -1 | head -5 | tail -1

commit 2daad1647a714b50dd696a02df3024c08be39585
    remove option to filter


In [16]:
hyperparameters = LstmBertNetworkFactoryhyperparameters_plain #BertNetworkFactoryhyperparameters_plain
inputs = plain_inputs # pub_inputs 


In [17]:
hyperparameters

{'accumulation_steps': '1',
 'batchsize': '64',
 'commit_id': '2daad1647a714b50dd696a02df3024c08be39585',
 'dataset': 'PpiAimedDatasetFactory',
 'earlystoppingpatience': 20,
 'epochs': '1000',
 'learningrate': 1e-05,
 'log-level': 'INFO',
 'network': 'RelationExtractorBertBiLstmNetworkNoPosFactory',
 'trainfile': 'AIMedFull.json'}

In [18]:
inputs

{'PRETRAINED_BIOBERT': 's3://aegovan-data/embeddings/bert/',
 'train': 's3://aegovan-data/aimed/AIMedFull.json'}

In [19]:
from sagemaker.pytorch import PyTorch

estimator = PyTorch(
     #entry_point='main_train_k_fold.py',
    entry_point='main_train_bert_k_fold.py',
                    source_dir = 'source/algorithms',
                    dependencies =['source/algorithms', 'source/datasets', 'source/preprocessor', 'source/modelnetworks','source/trainpipelinesbuilders'],
                    role=role,
                    framework_version ="1.0.0",
                    py_version='py3',
                    git_config= git_config,
                    image_name= docker_repo,
                    train_instance_count=1,
                    train_instance_type=instance_type,
                    hyperparameters =hyperparameters,
                    output_path=s3_output_path,
                    metric_definitions=metric_definitions,
                    #train_use_spot_instances = True
                    train_volume_size=30,
                    code_location=s3_code_path,
                    train_max_run = 60 * 60 * 24 * 4,
                    base_job_name ="aimed-ppi-bert-acc")

In [ ]:
estimator.fit(inputs)

2019-11-07 08:04:28 Starting - Starting the training job...
2019-11-07 08:04:42 Starting - Launching requested ML instances....